<a href="https://colab.research.google.com/github/Rob-A-B/wildfire-hotspots/blob/main/queimadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dask dask[dataframe] pyarrow

In [2]:
import os, datetime
from google.colab import files
import shutil, os, datetime
import os, glob, datetime
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import re
from google.colab import drive
import gc

import dask.dataframe as dd

In [3]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
hoje = datetime.date.today()
BASE   = "/content/projeto_pcb"
BRONZE_RAW = f"{BASE}/dados/bronze/raw"                   # onde guardamos os arquivos "brutos" por partição
BRONZE_CUR = f"{BASE}/dados/bronze/current"               # dataset consolidado (único arquivo)
os.makedirs(BRONZE_RAW, exist_ok=True)
os.makedirs(BRONZE_CUR, exist_ok=True)

# comentario teste de commit

# Define the path to your CSV files in Google Drive
# **IMPORTANT:** Replace 'My Drive/Your_Folder_With_CSVs' with the actual path to your folder
DRIVE_CSV_PATH = "/content/drive/My Drive/8_periodo/Big Data/datasets"
os.makedirs(DRIVE_CSV_PATH, exist_ok=True) # Create the directory if it doesn't exist

In [5]:
import os, datetime
from google.colab import files
import shutil, os, datetime
import os, glob, datetime
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import re
from google.colab import drive
import dask.dataframe as dd
import gc

# Mount Google Drive
drive.mount('/content/drive')

hoje = datetime.date.today()
BASE   = "/content/projeto_pcb"
BRONZE_RAW = f"{BASE}/dados/bronze/raw"                   # onde guardamos os arquivos "brutos" por partição
BRONZE_CUR = f"{BASE}/dados/bronze/current"               # dataset consolidado (único arquivo)
os.makedirs(BRONZE_RAW, exist_ok=True)
os.makedirs(BRONZE_CUR, exist_ok=True)

# Define the path to your CSV files in Google Drive
# **IMPORTANT:** Replace 'My Drive/Your_Folder_With_CSVs' with the actual path to your folder
DRIVE_CSV_PATH = "/content/drive/My Drive/8_periodo/Big Data/datasets" # Assuming this is the correct path from previous execution
os.makedirs(DRIVE_CSV_PATH, exist_ok=True) # Create the directory if it doesn't exist


# 1) Descobrir todos os CSVs diretamente em /content/drive/My Drive/Your_Folder_With_CSVs
arquivos_csv = [f for f in os.listdir(DRIVE_CSV_PATH) if f.endswith(".csv") and os.path.isfile(f"{DRIVE_CSV_PATH}/{f}")]
print(f"📦 Encontrados {len(arquivos_csv)} CSVs em {DRIVE_CSV_PATH}.\n")

# 2) Mover/copy to Bronze/raw partitioned and read into pandas DataFrames
for nome in arquivos_csv:
    caminho_src = f"{DRIVE_CSV_PATH}/{nome}"

    # extrai AAAAMM do nome (ex.: focos_mensal_br_202311.csv)
    m = re.search(r'(\d{6})', nome)
    aaaamm = m.group(1) if m else f"{hoje.year}{hoje:%m}"
    ano, mes = aaaamm[:4], aaaamm[4:]

    # destino particionado (save as parquet in bronze raw)
    dest_dir = os.path.join(BRONZE_RAW, f"ano={ano}", f"mes={mes}")
    os.makedirs(dest_dir, exist_ok=True)
    nome_parquet = nome.replace(".csv", ".parquet")
    caminho_dst_parquet = os.path.join(dest_dir, nome_parquet)


    # lê o CSV com pandas
    try:
        df_temp = pd.read_csv(caminho_src)
        df_temp["origem_arquivo"] = nome

        # salva como Parquet em bronze raw
        df_temp.to_parquet(caminho_dst_parquet, index=False)
        print(f"✅ Processado e salvo como Parquet: {nome}  ➜  {caminho_dst_parquet}")

        del df_temp # Free up memory
        gc.collect() # Run garbage collection

    except Exception as e:
        print(f"⚠️ Erro ao processar {caminho_src}: {e}")


# 3) Consolidate everything into a single Dask DataFrame (Bronze/current) by reading the Parquet files
# Read all parquet files from the raw bronze directory into a single Dask DataFrame
bronze_ddf = dd.read_parquet(os.path.join(BRONZE_RAW, "*/*/*.parquet"))

# Simple idempotence: remove exact duplicate rows
# (If desired, change to subset=["id","datahora","latitude","longitude"] if they exist)
bronze_ddf = bronze_ddf.drop_duplicates()

print(f"\n🧱 Bronze consolidated: {len(bronze_ddf):,} linhas, {bronze_ddf.shape[1]} colunas")
# For Dask, we use .head() to see the first few rows. This triggers computation.
display(bronze_ddf.head())


# 4) Salva consolidado (Parquet)
# We will save the consolidated data as a single Parquet file for efficiency with Dask
out_parquet = os.path.join(BRONZE_CUR, "focos_bronze_consolidado.parquet")

# To save as a single file, we repartition to 1 partition before saving
bronze_ddf.repartition(npartitions=1).to_parquet(out_parquet)


print(f"\n💾 Consolidado salvo em:\n - {out_parquet}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📦 Encontrados 22 CSVs em /content/drive/My Drive/8_periodo/Big Data/datasets.

✅ Processado e salvo como Parquet: focos_mensal_br_202504.csv  ➜  /content/projeto_pcb/dados/bronze/raw/ano=2025/mes=04/focos_mensal_br_202504.parquet
✅ Processado e salvo como Parquet: focos_mensal_br_202501.csv  ➜  /content/projeto_pcb/dados/bronze/raw/ano=2025/mes=01/focos_mensal_br_202501.parquet
✅ Processado e salvo como Parquet: focos_mensal_br_202502.csv  ➜  /content/projeto_pcb/dados/bronze/raw/ano=2025/mes=02/focos_mensal_br_202502.parquet
✅ Processado e salvo como Parquet: focos_mensal_br_202503.csv  ➜  /content/projeto_pcb/dados/bronze/raw/ano=2025/mes=03/focos_mensal_br_202503.parquet
✅ Processado e salvo como Parquet: focos_mensal_br_202505.csv  ➜  /content/projeto_pcb/dados/bronze/raw/ano=2025/mes=05/focos_mensal_br_202505.parquet
✅ Processado e salvo como Parquet: fo

,id,lat,lon,data_hora_gmt,satelite,municipio,estado,pais,municipio_id,estado_id,pais_id,numero_dias_sem_chuva,precipitacao,risco_fogo,bioma,frp,origem_arquivo,ano,mes
37,3283c9ee-3a7e-3123-9c39-b2b83631915c,-18.00720,-57.401901,2024-01-01 01:11:32,METOP-C,CORUMBÁ,MATO GROSSO DO SUL,Brasil,5003207,50,33,0.0,10.58,-999.00,Pantanal,NaN,focos_mensal_br_202401.csv,2024,1
44,1abcaee1-bd65-38f7-b12b-32586fe3953f,3.24090,-60.763302,2024-01-01 01:17:30,METOP-C,BOA VISTA,RORAIMA,Brasil,1400100,14,33,8.0,0.00,1.00,Amazônia,NaN,focos_mensal_br_202401.csv,2024,1
51,66778b43-fcc3-36f7-8dcc-50a09854b6db,-18.06290,-57.372100,2024-01-01 02:56:48,GOES-16,CORUMBÁ,MATO GROSSO DO SUL,Brasil,5003207,50,33,0.0,14.42,0.00,Pantanal,55.9,focos_mensal_br_202401.csv,2024,1
70,cc6ebd2c-100b-399b-b635-60ce4844791e,-12.55343,-41.485350,2024-01-01 04:01:00,NOAA-21,PALMEIRAS,BAHIA,Brasil,2923506,29,33,8.0,2.57,0.51,Caatinga,2.0,focos_mensal_br_202401.csv,2024,1
74,0df506db-cc03-361e-b1a6-bf5d11f0ac16,-12.56825,-41.455360,2024-01-01 04:01:00,NOAA-21,LENÇÓIS,BAHIA,Brasil,2919306,29,33,9.0,1.58,0.68,Caatinga,0.7,focos_mensal_br_202401.csv,2024,1



💾 Consolidado salvo em:
 - /content/projeto_pcb/dados/bronze/current/focos_bronze_consolidado.parquet


In [16]:
bronze_ddf.info()

Dask DataFrame dtypes:


,0
id,string[pyarrow]
lat,float64
lon,float64
data_hora_gmt,string[pyarrow]
satelite,string[pyarrow]
municipio,string[pyarrow]
estado,string[pyarrow]
pais,string[pyarrow]
municipio_id,int64
estado_id,int64



Dask DataFrame shape:


(<dask_expr.expr.Scalar: expr=DropDuplicates(frame=ReadParquetFSSpec(a12e366)).size() // 19, dtype=int64>,
 19)

In [6]:
bronze_ddf.isna().sum().compute()

,0
id,0
lat,0
lon,0
data_hora_gmt,0
satelite,0
municipio,0
estado,0
pais,0
municipio_id,0
estado_id,0


In [7]:
hoje = datetime.date.today()
BASE = "/content/projeto_pcb"
BRONZE = "/content/projeto_pcb/dados/bronze/current"
SILVER = "/content/projeto_pcb/dados/silver"
os.makedirs(SILVER, exist_ok=True)

In [8]:
df_temp = None
del df_temp
gc.collect()

41

In [9]:
bronze_ddf = bronze_ddf.drop(columns=['id', 'lat','lon','municipio_id','estado_id','pais_id','pais','origem_arquivo', 'ano', 'mes'])

In [ ]:
import os, glob, datetime
import pandas as pd
import numpy as np
import dask.dataframe as dd # Import dask

paths = sorted(glob.glob(os.path.join(BRONZE, "*.parquet"))) # Read parquet instead of csv
assert paths, "Nenhum arquivo Parquet encontrado em bronze."

# Read the consolidated parquet file into a Dask DataFrame
bronze_ddf = dd.read_parquet(paths[0]) # Assuming the consolidated file is the first one

esperadas = {"data_hora_gmt","satelite","municipio","estado"}
# Check for missing columns (Dask DataFrames have columns attribute)
faltando = esperadas - set(bronze_ddf.columns)
assert not faltando, f"Faltam colunas no dataset: {faltando}"

# Convert to datetime using Dask's astype or map_partitions with pandas to_datetime
bronze_ddf["data_hora_gmt"] = bronze_ddf["data_hora_gmt"].astype('datetime64[ns, UTC]')


for c in ["satelite","municipio","estado"]:
    # Use Dask's .str accessor for string operations
    bronze_ddf[c] = bronze_ddf[c].astype(str).str.strip()


# Drop rows with NaN in 'data_hora_gmt' using Dask's dropna
bronze_ddf = bronze_ddf.dropna(subset=["data_hora_gmt"])

# Identify and replace -999 with NaN in numeric columns
numeric_cols = bronze_ddf.select_dtypes(include=np.number).columns
# Use Dask's assign with a dictionary comprehension for replacement
replace_dict = {col: bronze_ddf[col].replace(-999, np.nan) for col in numeric_cols}
bronze_ddf = bronze_ddf.assign(**replace_dict)


# Calculate and print proportion of NaNs per column using Dask
nan_proportion_per_column = bronze_ddf.isnull().sum().compute() / len(bronze_ddf) * 100
print("\nProporção de NaN por coluna (%):")
print(nan_proportion_per_column)

# Calculate and print proportion of rows with NaN using Dask
rows_with_nan = bronze_ddf.isnull().any(axis=1).sum().compute()
proportion_rows_with_nan = rows_with_nan / len(bronze_ddf) * 100
print(f"\nProporção de linhas com NaN no total do dataset (%): {proportion_rows_with_nan:.2f}%")


# Drop rows with NaN values using Dask's dropna
bronze_ddf = bronze_ddf.dropna()

# Extract date components using Dask's .dt accessor
bronze_ddf["ano"] = bronze_ddf["data_hora_gmt"].dt.year.astype("Int64")
bronze_ddf["mes"] = bronze_ddf["data_hora_gmt"].dt.month.astype("Int64")
bronze_ddf["dia"] = bronze_ddf["data_hora_gmt"].dt.day.astype("Int64")

# carga que prof pediu
bronze_ddf["dt_carga"] = datetime.date.today().isoformat()

# Salva silver as Parquet
silver_parquet = os.path.join(SILVER, f"focos_silver_{datetime.date.today():%Y%m%d}.parquet")
os.makedirs(SILVER, exist_ok=True)
# Save as a single parquet file
bronze_ddf.repartition(npartitions=1).to_parquet(silver_parquet)




print("Silver salvo:", silver_parquet, "| linhas:", len(bronze_ddf))
# Use head() for Dask DataFrames to display the first few rows
display(bronze_ddf.head())


Proporção de NaN por coluna (%):
id                       0.000000
lat                      0.000000
lon                      0.000000
data_hora_gmt            0.000000
satelite                 0.000000
municipio                0.000000
estado                   0.000000
pais                     0.000000
municipio_id             0.000000
estado_id                0.000000
pais_id                  0.000000
numero_dias_sem_chuva    1.916553
precipitacao             1.180835
risco_fogo               2.209782
bioma                    0.000038
frp                      3.512460
origem_arquivo           0.000000
ano                      0.000000
mes                      0.000000
dtype: float64

Proporção de linhas com NaN no total do dataset (%): 6.23%


In [19]:
bronze_df.isna().sum()

,0
data_hora_gmt,0
satelite,0
municipio,0
estado,0
numero_dias_sem_chuva,302472
precipitacao,302472
risco_fogo,302472
bioma,9
frp,511105
ano,0


In [ ]:
bronze()

In [ ]:
GOLD = "/content/projeto_pcb/dados/gold
os.makedirs(GOLD, exist_ok=True)

In [ ]:
padrao_silver = "focos_silver_*.csv"   # ajuste se usou outro nome

paths = sorted(glob.glob(os.path.join(SILVER, padrao_silver)))
assert paths, f"Nenhum CSV Silver encontrado em: {SILVER}"

# parse_dates  reforçandos a conversão em seguida
df = pd.concat((pd.read_csv(p, parse_dates=["data_hora_gmt"]) for p in paths),
               ignore_index=True)

# Garante que data_hora_gmt é datetime
df["data_hora_gmt"] = pd.to_datetime(df["data_hora_gmt"], errors="coerce", utc=True)

# Normalizações úteis para agregação
df["data"] = df["data_hora_gmt"].dt.date
df["ano"]  = df["data_hora_gmt"].dt.year
df["mes"]  = df["data_hora_gmt"].dt.month
df["hora"] = df["data_hora_gmt"].dt.hour

por_estado_mes = (
    df.groupby(["estado","ano","mes"])
      .size()
      .reset_index(name="qtd_focos")
      .sort_values(["ano","mes","qtd_focos"], ascending=[True,True,False])
)


por_municipio_dia = (
    df.groupby(["estado","municipio","data"])
      .size()
      .reset_index(name="qtd_focos")
      .sort_values(["data","qtd_focos"], ascending=[True,False])
)


por_satelite_mes = (
    df.groupby(["satelite","ano","mes"])
      .size()
      .reset_index(name="qtd_focos")
      .sort_values(["ano","mes","qtd_focos"], ascending=[True,True,False])
)

por_satelite_total = (
    df.groupby("satelite")
      .size()
      .reset_index(name="qtd_focos")
      .assign(perc=lambda x: (x["qtd_focos"] / x["qtd_focos"].sum())*100)
      .sort_values("qtd_focos", ascending=False)
      .round({"perc": 2})
)

df["lat_cell"] = df["lat"].round(2)
df["lon_cell"] = df["lon"].round(2)
grade_espacial = (
    df.groupby(["lat_cell","lon_cell"])
      .size()
      .reset_index(name="qtd_focos")
      .sort_values("qtd_focos", ascending=False)
)

# ------------------------------
# Salva em GOLD
# ------------------------------
os.makedirs(GOLD, exist_ok=True)
hoje = datetime.date.today().isoformat()

g1 = os.path.join(GOLD, f"gold_focos_por_estado_mes_{hoje}.csv")
g2 = os.path.join(GOLD, f"gold_focos_por_municipio_dia_{hoje}.csv")
g3 = os.path.join(GOLD, f"gold_focos_por_satelite_mes_{hoje}.csv")
g3b= os.path.join(GOLD, f"gold_focos_por_satelite_total_{hoje}.csv")
g4 = os.path.join(GOLD, f"gold_grade_espacial_{hoje}.csv")

por_estado_mes.to_csv(g1, index=False)
por_municipio_dia.to_csv(g2, index=False)
por_satelite_mes.to_csv(g3, index=False)
por_satelite_total.to_csv(g3b, index=False)
grade_espacial.to_csv(g4, index=False)

print("Gold salvo:")
print(" -", g1)
print(" -", g2)
print(" -", g3)
print(" -", g3b)
print(" -", g4)

try:
    ult_ano  = por_estado_mes["ano"].max()
    ult_mes  = por_estado_mes.query("ano == @ult_ano")["mes"].max()
    top_estados = (
        por_estado_mes
        .query("ano == @ult_ano and mes == @ult_mes")
        .nlargest(10, "qtd_focos")
    )
    print(f"\nTOP estados no mês mais recente ({ult_ano}-{ult_mes:02d}):")
    print(top_estados.to_string(index=False))
except Exception as e:
    print("\n[Aviso] Não foi possível imprimir TOP estados do mês mais recente:", e)

print("\n% por satélite (total):")
print(por_satelite_total.to_string(index=False))

Gold salvo:
 - /content/projeto_pcb/dados/gold/gold_focos_por_estado_mes_2025-10-09.csv
 - /content/projeto_pcb/dados/gold/gold_focos_por_municipio_dia_2025-10-09.csv
 - /content/projeto_pcb/dados/gold/gold_focos_por_satelite_mes_2025-10-09.csv
 - /content/projeto_pcb/dados/gold/gold_focos_por_satelite_total_2025-10-09.csv
 - /content/projeto_pcb/dados/gold/gold_grade_espacial_2025-10-09.csv

TOP estados no mês mais recente (2024-03):
            estado  ano  mes  qtd_focos
           RORAIMA 2024    3       1767
       MATO GROSSO 2024    3       1508
             BAHIA 2024    3        936
MATO GROSSO DO SUL 2024    3        562
             GOIÁS 2024    3        429
         SÃO PAULO 2024    3        282
      MINAS GERAIS 2024    3        260
         TOCANTINS 2024    3        132
              PARÁ 2024    3         98
           ALAGOAS 2024    3         93

% por satélite (total):
 satelite  qtd_focos  perc
  NOAA-20       1870 28.18
  NPP-375       1399 21.08
 NPP-375D      

In [ ]:
# cont = df["Grey"].value_counts().sort_index()
# plt.bar(cont.index.astype(str), cont.values)
# plt.title("Distribuição da classe Grey")
# plt.xlabel("Grey (0=não cinza, 1=cinza)")
# # plt.ylabel("n")
# plt.show()


In [ ]:
# agg = df.groupby("Grey")[["R","G","B"]].mean().reset_index()
# for ch in ["R","G","B"]:
#     plt.bar(agg["Grey"].astype(str), agg[ch], label=ch)
#     plt.title(f"Média do canal {ch} por Grey")
#     plt.xlabel("Grey")
#     plt.ylabel(f"Média {ch} (escala {'0–255' if df['R'].max()>1 else '0–1'})")
#     plt.show()


# Task
Explain how to integrate Dask with cuDF for GPU acceleration in Google Colab.

## Set up a gpu runtime

### Subtask:
Ensure your Colab notebook is using a GPU runtime.


## Install rapids

### Subtask:
Install the necessary RAPIDS libraries, including cuDF and Dask-cuDF.


**Reasoning**:
Install the necessary RAPIDS libraries (cuDF and dask-cuDF) using pip.



In [ ]:
!pip install cudf-cu12 dask-cudf-cu12 --extra-index-url=https://pypi.nvidia.com